## Read in separate raw data files and create files of individual azimuth scans
### A. Ordog, June 2022
### NOTE: please do not modify
#### July 07, 2022: added in/out directories, removed old code, added plot to check scan trimming
#### July 24, 2022: added polarized data products to output npy files for making daily maps
#### August 29, 2022: changed input file naming to match with survey phases

## Import packages and define survey day and file directory

In [ ]:
import dva_sdhdf_combine_v3
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from astropy.modeling import models, fitting

day ='39'
surveyphase = '3'

#### Change the directory to where the files are located" ####
dir_in = '/home2/DATA_AO/DVA_DATA/dva_phase3/survey_phase3_day'+day+'/'
#dir_out = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
dir_out = '/home2/DATA_AO/DVA_DATA/dva_phase3/survey_phase3_day'+day+'/'
#dir_out = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/first_attempt/'
##############################################################

## Read in the file listing azimuth scan start and stop times:

In [ ]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the data and store it in arrays:
with open(dir_out+'DVAsurvey_phase3_day0'+day+'.txt') as fp:
    for line in fp:       
        scan_id.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])
        
# Print out the scan numbers with their start and stop times:
for i in range(0,len(scan_id)):
    print(f"{scan_id[i]:04}",scan_start[i],scan_stop[i])

# Convert start and stop times to Modified Julian Day (MJD).
# This is needed for plotting and for selecting out data collected
# between particular times:
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd


## Make files for each azimuth scan:¶
### ***Note: skip this if files already made
##### grey = all data  
##### orange = RA changing too much 
##### green = outside of scan time
##### black = good

In [ ]:
%%time

imp.reload(dva_sdhdf_combine_v3)

for i in range(0,len(scan_id)):
#for i in range(20,len(scan_id)):
#for i in range(0,5):
    print('===============================')
    print('Making scan ',i+1,' out of ',len(scan_id),' (scan id =',scan_id[i],') for times:')
    print(scan_start[i],scan_stop[i])
    print('===============================')
    print('')
    outname = 'dva_survey_phase3_raw_'+f"{int(scan_id[i]):04}"
    dva_sdhdf_combine_v3.combine(dir_in,dir_out,scan_start[i],scan_stop[i],outname,freq_s=2,
                                 az_scan_trim=True,freq_avg=True)
    print('')

## Read in scan files and stitch them together:

In [ ]:
%%time

t_set = []
az_set = []
dec_set = []
ra_set = []
el_set = []
noise_set = []
trim_flag = []

scan0 = f"{scan_id[0]:04}"

# Use one of the scans to get the list of frequencies:
file = h5py.File(dir_out+'dva_survey_phase3_raw_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][::12]/1e6

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq)])
LL_set = np.empty([0,len(freq)])
reRL_set = np.empty([0,len(freq)])
imRL_set = np.empty([0,len(freq)])

# Loop through all the scans in the "scan_num" list:
for i in scan_id:
    print(i)
    # select the file:
    file = h5py.File(dir_out+'dva_survey_phase3_raw_'+f"{i:04}"+'.h5','r')
    print(file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    # Add the position and time data to the corresponding arrays:
    dec_set = np.concatenate([dec_set,dataset['metadata']['declination']])
    ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension']])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation']])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth']])
    t_set = np.concatenate([t_set,dataset['metadata']['utc']])
    noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state']])
    trim_flag = np.concatenate([trim_flag,dataset['metadata']['trim_scan_flag']])
    
    # Add the spectrometer power data to the corresponding arrays:
    RR_set = np.concatenate([RR_set,dataset['data'][:,0,::12]],axis=0)
    LL_set = np.concatenate([LL_set,dataset['data'][:,1,::12]],axis=0)
    reRL_set = np.concatenate([reRL_set,dataset['data'][:,2,::12]],axis=0)
    imRL_set = np.concatenate([imRL_set,dataset['data'][:,3,::12]],axis=0)
    
t_set_plt = Time(t_set, format='isot',scale='utc').mjd

## Quick check of scan trimming:

In [ ]:
fig1, axs = plt.subplots(1,1,figsize=(18,6))

axs.scatter(t_set_plt,ra_set,s=1,color='C0')
axs.scatter(t_set_plt[trim_flag==1],ra_set[trim_flag==1],s=5,color='blue')

ax2 = axs.twinx()
ax2.scatter(t_set_plt,az_set,s=1,color='C1')
ax2.scatter(t_set_plt[trim_flag==1],az_set[trim_flag==1],s=5,color='red')
ax2.set_ylabel('Azimuth (deg)',fontsize=12)
ax2.plot([t_set_plt[0],t_set_plt[-1]],[0,0],color='black',linewidth=0.5)
ax2.plot([t_set_plt[0],t_set_plt[-1]],[360,360],color='black',linewidth=0.5)
for i in range(0,len(scan_id)):
    ax2.text(scan_start_mjd[i]+1e-3,180,f"{scan_id[i]:04}",rotation='vertical',fontsize=14)

axs.tick_params(axis="x")
axs.tick_params(axis="y")
axs.set_xlim(t_set_plt[0],t_set_plt[-1])
axs.set_ylim(-5,30)
ax2.set_ylim(-10,370)
axs.set_xlabel('Time (UTC)',fontsize=14)
axs.set_ylabel('RA (hr)',fontsize=14)
axs.set_title('Survey phase '+surveyphase+' day '+day,fontsize=14)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')

plt.tight_layout()
plt.savefig(dir_out+'scans_phase'+surveyphase+'_day'+day+'.pdf')
plt.savefig('../DVA2/PLOTS/scans_phase'+surveyphase+'_day'+day+'.pdf')

## Make numpy files for daily data:
#### This is just to produce a quick daily 'map' of the sky at two frequencies with scans we have so far.
#### Code for making the map is in a separate notebook (daily_map.ipynb)

In [ ]:
ra_set_trim = ra_set.copy()
dec_set_trim = dec_set.copy()
az_set_trim = az_set.copy()
el_set_trim = el_set.copy()
t_set_trim = t_set.copy()
t_set_plt_trim = t_set_plt.copy()
noise_set_trim = noise_set.copy()
RR_set_trim = RR_set.copy()
LL_set_trim = LL_set.copy()
reRL_set_trim = reRL_set.copy()
imRL_set_trim = imRL_set.copy()

idxtrim = np.where(trim_flag == 1)[0]
#print(idxtrim)

ra_set_trim[idxtrim] = np.nan
dec_set_trim[idxtrim] = np.nan 
az_set_trim[idxtrim] = np.nan 
el_set_trim[idxtrim] = np.nan 
t_set_trim[idxtrim] = np.nan 
t_set_plt_trim[idxtrim] = np.nan 
noise_set_trim[idxtrim] = np.nan 
RR_set_trim[idxtrim,:] = np.nan 
LL_set_trim[idxtrim,:] = np.nan 
reRL_set_trim[idxtrim,:] = np.nan 
imRL_set_trim[idxtrim,:] = np.nan

df = freq[1]-freq[0]

freq_plt = 800. # in MHz
wf = np.where(abs(freq-freq_plt)<df)[0][0]
np.save('../DVA2/DATA/Daily_maps/survey_phase3_all_800_day'+day+'.npy',([RR_set_trim[:,wf],LL_set_trim[:,wf],
                                                                         reRL_set_trim[:,wf],imRL_set_trim[:,wf],
                                                                         ra_set_trim,dec_set_trim,az_set_trim,el_set_trim,
                                                                         t_set_trim,noise_set_trim,t_set_plt_trim]))
freq_plt = 400. # in MHz
wf = np.where(abs(freq-freq_plt)<df)[0][0]
np.save('../DVA2/DATA/Daily_maps/survey_phase3_all_408_day'+day+'.npy',([RR_set_trim[:,wf],LL_set_trim[:,wf],
                                                                         reRL_set_trim[:,wf],imRL_set_trim[:,wf],
                                                                         ra_set_trim,dec_set_trim,az_set_trim,el_set_trim,
                                                                         t_set_trim,noise_set_trim,t_set_plt_trim]))
print(day)

In [ ]:
print(freq[-1]-freq[0])
print(len(freq))